# Twitter Scrape

Traversing a whole lot of shit...

In [1]:
# Libraries
import pandas as pd
import numpy as np
import sys, os
from time import time

from sklearn.externals import joblib

from multiprocessing import Pool

In [2]:
# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

# Not Helpful, keeping for (maybe) future
#from ldig.predict_tweet import *
#detector = Detector('../src/ldig/models/model.latin/')

In [3]:
# Directories
ext_dir = '../data/external/'
proc_dir = '../data/processed/'

scrape_in = ext_dir + 'scrape/'
scrape_out = proc_dir + 'scrape/3-31/{}-extract.csv'

In [4]:
# cols to keep
cols_scrape = [
    'username',
    'longitude',
    'latitude',
    'date',
    'message',
    'tweetID',
    'language',
    'hashtag1',
    'hashtag2',
    'hashtag3',
    'hashtag4',
    'hashtag5'
]
cols_out = [
    'username',
    'longitude',
    'latitude',
    'date',
    'message',
    'hashtag1',
    'hashtag2',
    'hashtag3',
    'hashtag4',
    'hashtag5'
]

In [5]:
# Classifier
clf = joblib.load('../data/processed/class/3-28/model.pkl')

# to prepare us for processing raw tweets
%run ../src/twitter-sentiment/preprocessing/__init__.py

In [6]:
# List of all the files we'll be evaluating

# testing
#scrape_f = os.listdir(scrape_in)

# 'production'
scrape_f = [x for x in  os.listdir(scrape_in)
           if (x.split('.')[0][-4:] != '_log')]

In [7]:
def process_cleaning(tweet):
    try:
        return process_one(tweet)
    except:
        #print (tweet)
        return None

def process_stemming(tweet):
    try:
        return stem_one(tweet)
    except:
        #print (tweet)
        return None
    
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, 10)
    pool = Pool(4)
    #df = pd.concat(pool.map(func, df_split))
    df = [pool.map(func, df_split)]
    pool.close()
    pool.join()
    return df

def parallelize_series(series, func):
    pool = Pool(6)

    df = pool.map(func, series)

    pool.close()
    pool.join()
    return df

In [8]:
def process_scrape_csv(csv_f, gps=False, verbose=False):
    """
    """
    if (verbose):
        # Set a pattern for verbosity
        metrics = {'f_name': csv_f}
        start = time()
        print (csv_f)
    
    # read in
    f_ = csv_f
    df = pd.read_csv(f_, usecols=cols_scrape, dtype={'message': str})
    if (verbose):
        metrics['orig'] = df.shape[0]
        print ("Number of tweets\n\tOriginal:\t\t{}".format(metrics['orig']))
        
    # pre screen our messages
    df.dropna(subset=['message'], inplace=True)
    if (verbose):
        metrics['message'] = df.shape[0]
        print ("\tWith messages:\t\t{}".format(metrics['message']))
        
    # filter, keep only english tweets
    df = df[df.language == 'en']
    if (verbose):
        metrics['eng'] = df.shape[0]
        print ("\tLang (twit.) eng.:\t{}".format(metrics['eng']))
    
    # Predict language
    ## Tested and it doesn't help...
    #df['pred_lang'] = parallelize_series(df.message.values, detector.get_tweet_lang)
    #english = df[df.pred_lang == 'en']
    #if (verbose):
        #print ("\tLang (class.) eng.:\t\t\t{}".format(df.shape[0]))
    
    # tokenize
    df['clean'] = parallelize_series(df.message.values, process_cleaning)
    df = df[pd.notnull(df['clean'])]
    if (verbose):
        metrics['clean'] = df.shape[0]
        print ("\tWith viable tokens:\t{}".format(metrics['clean']))
        
    # stem
    df['stem'] = parallelize_series(df.clean.values, process_stemming)
    df = df[pd.notnull(df['stem'])]
    if (verbose):
        metrics['stem'] = df.shape[0]
        print ("\tWith stems:\t\t{}".format(metrics['stem']))
    
    # CLASSIFY
    df['label'] = clf.predict(df.stem.values)
    df = df[df.label == 1]
    if (verbose):
        metrics['label'] = df.shape[0]
        print ("\tClassified+:\t\t{}".format(metrics['label']))
        
    # Filter to just those with GPS coordinates
    if (gps):
        df = df[pd.notnull(df['longitude'])]
        if (verbose):
            metrics['gps'] = df.shape[0]
            print ('\tGPS:\t\t\t')
            
        
    #if verbose, lets log that out
    if (verbose):
        secs = time() - start
        metrics['time'] = secs
        print ("... in {} seconds\n".format(secs))
        
        log.append(metrics)
    
    return df

In [ ]:
# Keep a dataframe for a log
#log_df = pd.DataFrame(columns=['f_name', 'time', 'orig', 'message', 'eng', 'clean', 'stem', 'label'])
log = []

In [ ]:
for f in scrape_f:
    
    # file names
    f_in = scrape_in + f
    f_out = scrape_out.format(f[:-4])
    f_gps = f_out[:-4] + '-gps.csv'
    
    # sanity check for times sake
    if (os.path.isfile(f_in) == False):
        continue
    else:
        try:
            # read in new tweets
            labeled_df = process_scrape_csv(f_in, gps=True, verbose=True)
            labeled_df.to_csv(f_lat, index=False)
            
            # write out the now filtered csv
            #labeled_df.to_csv(out_f, index=False)
            #labeled_df[pd.notnull(labeled_df['longitude'])].to_csv(f_lat, columns=cols_out, index=False)
        
        except:
            log.append({'f_name': f, 'orig': None})
            print ('Could not read.\n')

../data/external/scrape/tweets_immigrant_34315.csv
Number of tweets
	Original:		274900
	With messages:		274215
	Lang (twit.) eng.:	141850
	With viable tokens:	141850
	With stems:		141849
	Classified+:		1483
	GPS:			
... in 14.60554552078247 seconds

Could not read.

../data/external/scrape/tweets_immigrant_34316.csv
Number of tweets
	Original:		375600
	With messages:		374678
	Lang (twit.) eng.:	180172
	With viable tokens:	180172
	With stems:		180170
	Classified+:		2341
	GPS:			
... in 18.990891695022583 seconds

Could not read.

../data/external/scrape/tweets_immigrant_34317.csv
Number of tweets
	Original:		358100
	With messages:		357266
	Lang (twit.) eng.:	172860
	With viable tokens:	172860
	With stems:		172860
	Classified+:		2237
	GPS:			
... in 18.01806139945984 seconds

Could not read.

../data/external/scrape/tweets_immigrant_34318.csv
Number of tweets
	Original:		355200
	With messages:		354354
	Lang (twit.) eng.:	174685
	With viable tokens:	174685
	With stems:		174681
	Classified

In [ ]:
log_df = pd.DataFrame.from_records(log)
log_df.to_csv('../data/processed/scrape/3-31/log.csv', index=False)